
# TiO₂ surfaces for water splitting — reproducible notebook

**What this notebook contains (complete, reproducible workflow):**
- Brief theory and *why* we choose specific surfaces for water splitting.
- Code to fetch bulk TiO₂ structures from **Materials Project** (or fallback to local CIFs).
- Code to build slabs for **rutile (110)** and **anatase (101)** and **anatase (001)**, relax them (placeholder) and save **POSCARs** for VASP.
- Example `INCAR` and `KPOINTS` templates and suggestions for DFT parameters.
- Extensive inline comments and Markdown explanations for every step — so you can follow and reproduce.

**Notes before running:**
- To download structures from Materials Project you need an API key. Place it in the environment variable `PMG_MAPI_KEY` or set `MP_API_KEY` variable in the notebook cells.
- This notebook *does not* attempt to run VASP. It prepares structures and inputs. You should run DFT on your HPC cluster.
- If you do not have a Materials Project API key, the notebook will show how to use local CIFs instead.



## Why these surfaces? (short justification)

**Rutile (110)** — *benchmark & surface science reference*
- Rutile is the thermodynamically stable TiO₂ polymorph at bulk conditions. The (110) surface is the most commonly studied rutile facet in surface science because it is the most stable and well-characterized experimentally (STM, LEED, TPD, XPS).
- For water splitting, rutile(110) is crucial as a **reference**: many mechanistic and adsorption studies use it as a baseline. If you want to compare to older experimental/theoretical literature, model this surface.

**Anatase (101)** — *practical photocatalysis standard*
- Anatase nanoparticles commonly expose the (101) facet and are known to show strong photocatalytic performance (good charge separation). This facet tends to adsorb molecular water and is a representative "realistic" exposed facet of nanomaterials used in water splitting experiments.
- Use anatase(101) when your goal is to model typical photocatalytic nanoparticles and overall water splitting activity.

**Anatase (001)** — *reactive facet, complementary study*
- Anatase(001) is more reactive than (101) (higher surface energy), often implicated in oxidation reactions (OER). It's less stable but highly active — useful as a complementary surface to study high-activity sites for OER intermediates.

**Practical modelling note:** water splitting involves both hydrogen evolution (HER) and oxygen evolution (OER). Often the **reduction** (HER) benefits from good electron transport (anatase or rutile), while **oxidation** (OER) is favored on higher-energy, more reactive facets (like anatase(001)). Therefore we include both anatase(101) and anatase(001) plus rutile(110) as a benchmark.


In [9]:

# Code: imports, helper functions and environment checks.
# This cell tries to import necessary python packages and defines helper functions.
# If you run this notebook, please ensure you have 'pymatgen' and 'ase' installed in your environment.
# Install with: pip install pymatgen ase
from pathlib import Path
import os
from pymatgen.core.surface import SlabGenerator, generate_all_slabs
from pymatgen.core.structure import Structure
from pymatgen.ext.matproj import MPRester

OUTDIR = Path("TiO2_surfaces_output")
OUTDIR.mkdir(exist_ok=True)
print("Output directory:", OUTDIR.resolve())

def save_poscar(structure, filename):
    """Save POSCAR using pymatgen's Structure.to with VASP format."""
    with open(filename, 'w') as f:
        f.write(structure.to(fmt="poscar"))
    print(f"Saved POSCAR: {filename}")

def make_and_save_slab(bulk_struct, miller, min_slab_thickness=10.0, min_vacuum_thickness=15.0, center_slab=True, filename_prefix="slab"):
    """Generate a slab and save POSCAR. Returns first slab object."""
    sg = SlabGenerator(initial_structure=bulk_struct, miller_index=miller,
                       min_slab_size=min_slab_thickness, min_vacuum_size=min_vacuum_thickness, center_slab=center_slab)
    slabs = sg.get_slabs()  # list of slabs; we take the first (usually symmetric) slab
    if not slabs:
        raise ValueError(f"No slab generated for {miller}")
    slab = slabs[0]
    save_poscar(slab, OUTDIR / f"{filename_prefix}_{miller[0]}{miller[1]}{miller[2]}_POSCAR.vasp")
    return slab


Output directory: /home/Mayday3003/Documents/Github-repos/MXene_Ti3O2T_watter_splitting_aplication/book/surface/TiO2_surfaces_output



## Materials Project access (automatic) or local CIF fallback

The following cell will try to fetch bulk structures from Materials Project using `MPRester`.
- **If you have an API key**, set it in the environment variable `PMG_MAPI_KEY` (or `MP_API_KEY`) before running the cell.
- **If not**, the cell will instruct you how to place CIF files in the `./cifs/` directory and will load them from there.

We target **mp-2657 (rutile)** and **mp-390 (anatase)** which are the commonly used bulk entries in Materials Project.


In [10]:

# Attempt to fetch bulk structures from Materials Project; fallback to local CIFs.
from pymatgen.ext.matproj import MPRester
import os
local_cif_dir = Path("materials")
local_cif_dir.mkdir(exist_ok=True)

MP_API_KEY = 'FHqPoi7BeE4MqQ5eUt9csKC55KgoiiCZ'
# Materials Project IDs commonly used
mpids = {
    'rutile': 'mp-2657',  # rutile TiO2
    'anatase': 'mp-390'   # anatase TiO2
}

structures = {}

api_key = os.environ.get('PMG_MAPI_KEY') or os.environ.get('MP_API_KEY') or None
if api_key:
    print("Materials Project API key detected; trying to fetch structures...")
    try:
        with MPRester(api_key) as mpr:
            for name, mid in mpids.items():
                print(f"Fetching {name} -> {mid}")
                struct = mpr.get_structure_by_material_id(mid)
                structures[name] = struct
                print(f"Fetched {name}: formula {struct.formula}, lattice {struct.lattice}")
    except Exception as e:
        print("Failed to fetch from Materials Project:", e)
        api_key = None

if not api_key:
    # fallback: look for local CIFs: anatase.cif and rutile.cif in ./cifs/
    print("No Materials Project fetch. Looking for local CIF files in ./cifs/") 
    fallback_files = {
        'rutile': local_cif_dir / 'rutile.cif',
        'anatase': local_cif_dir / 'anatase.cif'
    }
    for name, p in fallback_files.items():
        if p.exists():
            print(f"Loading local CIF for {name}: {p}")
            structures[name] = Structure.from_file(p)
        else:
            print(f"Local CIF not found for {name}: expected {p}; please add it or set PMG_MAPI_KEY.")

# summary
for k,v in structures.items():
    print(k, v.composition, v.lattice)


No Materials Project fetch. Looking for local CIF files in ./cifs/
Local CIF not found for rutile: expected materials/rutile.cif; please add it or set PMG_MAPI_KEY.
Local CIF not found for anatase: expected materials/anatase.cif; please add it or set PMG_MAPI_KEY.



## Generate slabs for the chosen surfaces

We will generate:
- **rutile (110)** from the rutile bulk (mp-2657)
- **anatase (101)** and **anatase (001)** from anatase bulk (mp-390)

The slab generator parameters chosen (`min_slab_size` and `min_vacuum_size`) are conservative defaults:
- slab thickness ~10 Å ensures several atomic layers (tune for convergence),
- vacuum ≥15 Å avoids slab-slab interactions in z.


In [11]:

# Generate slabs for rutile(110), anatase(101) and anatase(001)
slabs_generated = {}
if 'rutile' in structures:
    print('Making rutile (110) slab...')
    slabs_generated['rutile_110'] = make_and_save_slab(structures['rutile'], (1,1,0), min_slab_thickness=12.0, min_vacuum_thickness=18.0, filename_prefix='rutile')
if 'anatase' in structures:
    print('Making anatase (101) slab...')
    slabs_generated['anatase_101'] = make_and_save_slab(structures['anatase'], (1,0,1), min_slab_thickness=12.0, min_vacuum_thickness=18.0, filename_prefix='anatase_101')
    print('Making anatase (001) slab...')
    slabs_generated['anatase_001'] = make_and_save_slab(structures['anatase'], (0,0,1), min_slab_thickness=12.0, min_vacuum_thickness=18.0, filename_prefix='anatase_001')

print('Done. Generated slabs (keys):', list(slabs_generated.keys()))


Done. Generated slabs (keys): []



## Example INCAR and KPOINTS templates for VASP

Below are suggested templates you can use as starting points. **Adjust** cutoffs, U values, k-points and smearing to match your code/pseudopotentials/HPC policy.


In [12]:

# Write example INCAR and KPOINTS files to OUTDIR
incar_text = """SYSTEM = TiO2 surface (example)
ENCUT = 520
ISMEAR = 0
SIGMA = 0.05
EDIFF = 1E-5
EDIFFG = -0.02
IBRION = 2
NSW = 100
ISIF = 2
LREAL = Auto
LWAVE = .FALSE.
LCHARG = .FALSE.
# For improved energies of defect states/band edges consider HSE06 or PBE+U
# For PBE+U example:
# LDAU = .TRUE.
# LDAUTYPE = 2
# LDAUL = 2 -1
# LDAUU = 4.2 0.0   # Ti U ~ 4 eV (tune as needed)
# LDAUJ = 0 0
"""

kpoints_text = """Automatic mesh
0
Monkhorst
4 4 1
0 0 0
"""

with open(OUTDIR / 'INCAR.example', 'w') as f:
    f.write(incar_text)
with open(OUTDIR / 'KPOINTS.example', 'w') as f:
    f.write(kpoints_text)
print('Wrote example INCAR and KPOINTS to', OUTDIR) 


Wrote example INCAR and KPOINTS to TiO2_surfaces_output



---

### What I did and why (summary inside the notebook)
- Picked **rutile (110)** for benchmarking and comparison with surface-science literature. It's stable and has deep experimental coverage.
- Picked **anatase (101)** as the standard photocatalytic exposed facet (representative for nanoparticles used in photocatalysis).
- Picked **anatase (001)** as a complementary reactive facet often implicated in OER steps.
- Prepared a reproducible notebook that **fetches bulk structures (mp-2657 and mp-390)** from Materials Project (if API key available), falls back to local CIFs otherwise, builds slabs with conservative thickness and vacuum, saves POSCARs, and writes example VASP input templates.

You can now run the notebook on your machine (or HPC interactive node). If you want, I can also:
- prepare further automated convergence tests (slab thickness, k-points),
- add automated Bader charge analysis scripts,
- generate water adsorption initial structures (H2O placements) for reaction path sampling,
- or produce NEB templates for OER/HER steps.
